In [1]:
import numpy as np
import scipy.io as sio
import math
import scipy
from scipy import optimize
import pandas as pd
import matplotlib.pyplot as plt
import latexify
from scipy.integrate import odeint
import timeit
import numba
from numba import jit

C:\Users\ufkug\.conda\envs\env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ufkug\.conda\envs\env\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\ufkug\.conda\envs\env\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [15]:

# Initialize the values stated at Pehlevan's Paper
alpha = 0.3
k = 32
l1 = 0.3
l2 = 0.1
b = np.random.uniform(0,1,k)
w_x = np.random.uniform(1,5,k)
v_m = np.random.uniform(0,1/math.sqrt(k),(k,k))
m = v_m.dot(v_m.transpose())
dt = 0.01  # Time steps for Euler
tau = 500  # Overall time
I = w_x - l1*np.ones(k) - alpha*b
V = np.zeros(k)
spike = np.zeros(k) # Stores spikes at the moment
thresh = np.zeros(k) # Stores threshold voltages
y = np.zeros(k)
holding = np.zeros(k) # Stores spike counts




# Test the convergence of spiking neural dynamics
for step in np.arange(dt,tau,dt):
    
    V += I*dt
    thresh = l2 + np.diag(m)
    
    # Find indexes of voltages exceeding the threshold
    index = np.greater(V,thresh)
    
    # Set exceeding voltages to 0, launch a spike
    V[index] = 0
    holding[index] += 1
    spike[index] = 1/dt
    
    # If there is no spike, set 0
    spike[~index] = 0
    
    m_nodiag = m - np.diag(np.diag(m))
    I += (w_x - I - l1 - alpha*b - m_nodiag @ spike)*dt

y = holding/step
h = -2*y @ (w_x-alpha*b) + 2*l1*np.linalg.norm(y,ord=1) + l2*((np.linalg.norm(y,ord=2))**2) + y@m@y
print(y)

[0.00000000e+00 0.00000000e+00 2.00004000e-03 2.00004000e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.73803476e+00
 2.20204404e+00 0.00000000e+00 0.00000000e+00 2.00004000e-03
 2.00004000e-03 2.00004000e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 4.58209164e+00 0.00000000e+00 0.00000000e+00
 2.00004000e-03 0.00000000e+00 0.00000000e+00 3.14606292e+00
 0.00000000e+00 5.08010160e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]


In [16]:
w_x.shape

(32,)

In [17]:
print(h)

-71.73399902316574


In [18]:
def f(t,m = m,w_x = w_x,l1 = l1,l2 = l2):
    t_trans = np.transpose(t)
    return -2*t.dot(w_x-alpha*b) + 2*l1*np.linalg.norm(t,ord=1) + l2*(np.linalg.norm(t))**2 + t.dot(m.dot(t))
r = optimize.minimize(f, np.zeros(k), bounds = [(0,None)]*k)
r

      fun: -71.76669604391522
 hess_inv: <32x32 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 6.76320866e+00,  6.12174773e+00,  8.96169183e-01,  2.31558772e+00,
        5.15621537e+00,  5.94528444e+00,  4.08616216e+00,  4.26325644e-06,
        4.26325644e-06,  2.62306656e+00,  2.22748326e+00,  1.94561522e+00,
        2.05368593e+00,  1.00379651e+00,  2.40911930e+00,  3.73205467e+00,
        5.13709466e+00,  4.26325644e-06,  6.34396713e+00,  4.81833382e+00,
        1.88061335e+00,  7.73015358e+00,  3.20188889e+00,  1.42108548e-06,
        2.23128183e+00, -1.42108548e-06,  2.47028424e+00,  3.65000261e+00,
        4.37056968e+00,  5.48557608e+00,  3.78471867e+00,  4.02294518e+00])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 792
      nit: 23
     njev: 24
   status: 0
  success: True
        x: array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.63814712, 2.1034625 , 0.        ,
       0.        , 0

In [19]:
np.linalg.norm(y-r['x'],ord=2)/np.linalg.norm(r['x'],ord=2)

0.031188613431296693

In [20]:
(abs(r['fun']-h))/abs(r['fun'])

0.0004556015889246616

In [32]:
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image

In [41]:
x = scipy.io.loadmat('whitened.mat')
x = x['IMAGES']
x = x[:,:,1]
x = np.array(x)
x_patches = image.extract_patches_2d(x, (16,16))
num, length = x_patches.shape[0], 256
x_vectors = np.zeros((num,length))
# Take 16x16 patches:
for i in range(x_patches.shape[0]):
    x_vectors[i] = x_patches[i].flatten()
x = x_vectors.T

In [44]:
x.shape[1]

247009

In [7]:
l = np.random.uniform(0,1,(10,200000))

In [27]:
x = A @ l[:,0:1000]

In [57]:
@jit(nopython = True)
def SNN(x):
    #learn_rate = 10**(-3)
    alpha = 0
    l1 = 0
    l2 = 0
    k = x.shape[0]
    n = x.shape[0]
    w = np.random.normal(0,1/196,(k,n))
    #w = w_first
    m = np.eye(k)
    b = np.zeros(k)
    dt = 0.01
    tau = 100
    time_max = x.shape[1]
    learn_rate = np.zeros(time_max)
    learn_rate[0:10**4] = (10**(-3))
    learn_rate[10**4:10**5] = 10**(-5)
    learn_rate[10**5:time_max] = 0.5 * 10**(-5)

    V = np.zeros(k)
    spike = np.zeros(k)
    thresh = np.zeros(k)
    y = np.zeros(k)
    y_hold = np.zeros((k,time_max))
    holding = np.zeros(k)
    m_nodiag = m - np.diag(np.diag(m))
    count = 0
    w_x = np.zeros(k)
    I = np.zeros(k)
    #a = 10
    #bi = 0.01

    # Full Algorithm
    for t in np.arange(0,time_max,1):
        #learn_rate = 1/(a+bi*t)
        w_x = w.dot(x[:,t])
        I = w_x  - l1*np.ones(k) - alpha*b
        holding = np.zeros(k)
        spike = np.zeros(k)
        V = np.zeros(k)
        thresh = l2 + np.diag(m)
        # Spiking neural dynamics
        for step in np.arange(dt,tau,dt):
            V += I*dt


            # Find indexes of voltages exceeding the threshold
            index = np.greater(V,thresh)

            # Set exceeding voltages to 0, launch a spike
            V[index] = 0
            holding[index] += 1
            spike[index] = 1/dt

            # If there is no spike, set 0
            spike[~index] = 0

            I += (w_x - I - l1 - alpha*b - m_nodiag.dot(spike))*dt

        y = holding/tau
        y_hold[:,t] = y
        # Synaptic and homeostatic plasticity
        learn_rate_now = learn_rate[t]
        w += learn_rate_now * (np.outer(y,x[:,t])- w)
        m += learn_rate_now * (np.outer(y,y) - m)
        b += learn_rate_now * (alpha*y-b)
        m_nodiag = m - np.diag(np.diag(m))
        count += 1
        print(count)

In [ ]:
SNN(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218

220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278

In [26]:
y_hold

[]

In [21]:
%timeit SNN(x)

<ipython-input-20-247b381b965d>:36: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 1d, A))
  w_x = w @ x[:,t]
C:\Users\ufkug\.conda\envs\env\lib\site-packages\numba\core\typing\npydecl.py:930: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 1d, A))
  warnings.warn(NumbaPerformanceWarning(msg))


656 ms ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit SNN.py_func(x)

10.4 s ± 33.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
np.testing.assert_array_equal(SNN(x), SNN.py_func(x))

C:\Users\ufkug\.conda\envs\env\lib\site-packages\numpy\testing\_private\utils.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x = array(x, copy=False, subok=True)


ValueError: could not broadcast input array from shape (10,10) into shape (10)

In [716]:
err_current = np.linalg.norm(x.transpose() @ x - y_hold.transpose() @ y_hold,ord = 'fro')**2 / (2*(time_max**2))
print(err_current)

1.9135363376936485


In [722]:
a = np.random.uniform(0,1,(n,time_max))
err_current = np.linalg.norm(x.transpose() @ x - a.transpose() @ a,ord = 'fro')**2 / (2*(time_max**2))
print(err_current)

63.6446506270249


In [721]:
def f(t,x = x,time_max = time_max):
    a = t.reshape((k,time_max))
    return np.linalg.norm(x.transpose() @ x - a.transpose() @ a,ord = 'fro')**2 / (2*(time_max**2))
r = optimize.minimize(f, np.ones(k*time_max), bounds = [(0,None)]*(k*time_max))
r

      fun: 0.8259281905666156
 hess_inv: <300x300 LbfgsInvHessProduct with dtype=float64>
      jac: array([-3.33066906e-07,  2.10942376e-07, -3.88578057e-07, -6.66133819e-08,
        2.77555758e-07,  6.66133819e-08, -2.44249067e-07, -2.99760215e-07,
       -2.99760215e-07, -5.44009285e-07, -5.44009279e-07, -1.11022303e-07,
       -2.44249067e-07, -3.66373600e-07, -8.88178425e-08,  6.66133819e-08,
       -5.77315970e-07, -5.77315970e-07, -1.77635683e-07, -3.66373596e-07,
        2.77555758e-07,  6.66133819e-08,  1.88737915e-07,  2.44249067e-07,
       -2.10942376e-07, -1.77635683e-07, -2.10942376e-07, -1.11022303e-07,
       -1.55431224e-07, -1.55431224e-07, -3.33066906e-07,  2.10942376e-07,
       -3.88578057e-07, -6.66133819e-08,  2.77555758e-07,  6.66133819e-08,
       -2.44249067e-07, -2.99760215e-07, -2.99760215e-07, -5.44009285e-07,
       -5.44009279e-07, -1.11022303e-07, -2.44249067e-07, -3.66373600e-07,
       -8.88178425e-08,  6.66133819e-08, -5.77315970e-07, -5.77315970e-07,

In [568]:
t = r['x']
t = t.reshape((k,time_max))
np.linalg.norm(y_hold-t,ord=2)/np.linalg.norm(t,ord=2)

0.8303589526267953

In [577]:
np.outer(y_hold[:,1],x[:,1])

array([[ 0.        , -0.        ,  0.        , -0.        ,  0.        ],
       [ 0.63480654, -0.47252423,  0.62327478, -0.31275045,  0.18269842],
       [ 0.        , -0.        ,  0.        , -0.        ,  0.        ],
       [ 0.        , -0.        ,  0.        , -0.        ,  0.        ],
       [ 0.82877521, -0.61690663,  0.81371986, -0.40831309,  0.23852294]])

      fun: 0.7488186827513797
 hess_inv: <100x100 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.95319325e-06,  4.88498131e-07,  2.64233079e-06,  1.88737913e-07,
        3.51940697e-06,  5.36237721e-06, -9.99200717e-07,  3.70814488e-06,
       -2.12052597e-06,  5.67452085e-02, -2.95319325e-06,  4.88498131e-07,
        2.64233079e-06,  1.88737913e-07,  3.51940697e-06,  5.36237721e-06,
       -9.99200717e-07,  3.70814488e-06, -2.12052597e-06,  5.67452085e-02,
       -2.95319325e-06,  5.10702592e-07,  2.64233079e-06,  1.88737913e-07,
        3.51940697e-06,  5.36237721e-06, -9.99200717e-07,  3.70814488e-06,
       -2.12052597e-06,  5.67452085e-02, -2.95319325e-06,  4.66293671e-07,
        2.64233079e-06,  1.88737913e-07,  3.51940697e-06,  5.32907052e-06,
       -1.01030295e-06,  3.70814488e-06, -2.12052597e-06,  5.67451752e-02,
       -2.95319325e-06,  4.66293671e-07,  2.64233079e-06,  1.88737913e-07,
        3.51940697e-06,  5.32907052e-06, -1.01030295e-06,  3.70814488e-06,

In [329]:
t = r['x']
t = t.reshape((k,time_max))
np.linalg.norm(x.transpose() @ x - t.transpose() @ t,ord = 'fro')**2 / (2*(time_max**2))

0.0851043944919449

In [330]:
print(t)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
x = sio.loadmat('x.mat')
y_hold = sio.loadmat('y_hold.mat')
x = x['x']
y_hold = y_hold['y_hold']

In [66]:
err = 0
for i in range(time_max):
    for j in range(time_max):
        err += ( x[:,i] @ x[:,j] - y_hold[:,i] @ y_hold[:,j])**2
    print(i)
err /= 2*(time_max)**2
print(err)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
0.8694865100233246


In [21]:
print(err_current/ (2*(time_max)**2))

NameError: name 'err_current' is not defined

4.918640025854357


In [260]:
print(l.transpose()[1])

[0.39355057 0.819897   0.88016036 0.12764565 0.51136838]


In [323]:
scipy.io.savemat('c:/Users/ufkug/x.mat', mdict={'x': x})
scipy.io.savemat('c:/Users/ufkug/y_hold.mat', mdict={'y_hold': y_hold})

In [126]:
scipy.io.savemat('c:/Users/ufkug/b.mat', mdict={'b': b})
scipy.io.savemat('c:/Users/ufkug/w_x.mat', mdict={'w_x': w_x})
scipy.io.savemat('c:/Users/ufkug/m.mat', mdict={'m': m})
scipy.io.savemat('c:/Users/ufkug/k.mat', mdict={'k': k})

In [193]:
output

[array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.04249344, 0.        ,
        0.        , 0.        , 0.        , 0.  

In [42]:
h=np.asmatrix(np.zeros(5))

In [43]:
h

matrix([[0., 0., 0., 0., 0.]])

In [44]:
np.array(h)[0]

array([0., 0., 0., 0., 0.])

In [ ]:
minim = math.inf
for a in [10,100,1000,10000]:
    for bi in [0.01, 0.1, 1]:
        #learn_rate = 10**(-3)
        alpha = 0
        l1 = 0
        l2 = 0
        k = x.shape[0]
        n = x.shape[0]
        w = np.random.normal(0,1/2,(k,n))
        #w = w_first
        m = np.eye(k)
        b = np.zeros(k)
        dt = 0.01
        tau = 100
        time_max = x.shape[1]
        #learn_rate = np.zeros(10**5)
        #learn_rate[0:10**4] = (10**(-1))
        #learn_rate[10**4:10**5] = 10**(-2)

        V = np.zeros(k)
        spike = np.zeros(k)
        thresh = np.zeros(k)
        y = np.zeros(k)
        y_hold = np.zeros((k,time_max))
        holding = np.zeros(k)
        m_nodiag = m - np.diag(np.diag(m))
        count = 0
        w_x = np.zeros(k)
        I = np.zeros(k)
        # Full Algorithm
        for t in np.arange(0,time_max,1):
            learn_rate = 1/(a+bi*t)
            w_x = w @ x[:,t]
            I = w_x  - l1*np.ones(k) - alpha*b
            holding = np.zeros(k)
            spike = np.zeros(k)
            V = np.zeros(k)
            thresh = l2 + np.diag(m)
            # Spiking neural dynamics
            for step in np.arange(dt,tau,dt):
                V += I*dt


                # Find indexes of voltages exceeding the threshold
                index = np.greater(V,thresh)

                # Set exceeding voltages to 0, launch a spike
                V[index] = 0
                holding[index] += 1
                spike[index] = 1/dt

                # If there is no spike, set 0
                spike[~index] = 0

                I += (w_x - I - l1 - alpha*b - m_nodiag @ spike)*dt

            y = holding/tau
            y_hold[:,t] = y
            # Synaptic and homeostatic plasticity
            w += learn_rate * (np.outer(y,x[:,t])- w)
            m += learn_rate * (np.outer(y,y) - m)
            b += learn_rate * (alpha*y-b)
            m_nodiag = m - np.diag(np.diag(m))
            count += 1
            #print(count)
        err_current = np.linalg.norm(x.transpose() @ x - y_hold.transpose() @ y_hold,ord = 'fro')**2 / (2*(time_max**2))
        if minim > err_current:
            minim = err_current
            a_min = a
            b_min = bi
print(a_min)
print(b_min)
print(minim)
